In [2]:
from sklearn import datasets
import numpy as np
from sklearn.linear_model import Ridge

In [3]:
filename = "/Users/yilanliu/Desktop/UC DAVIS/STA 141C/HW1/cpusmall.txt"
X,y = datasets.load_svmlight_file(filename)

In [4]:
from scipy import sparse

In [5]:
X = sparse.csr_matrix.todense(X)

In [6]:
from sklearn import preprocessing
X = preprocessing.scale(X)
y = preprocessing.scale(y)

In [7]:
initial_value = np.transpose(np.matrix(np.random.uniform(-0.05,0.05,12)))

In [8]:
y = np.transpose(y)
y = y.reshape(8192,1)

In [9]:
def gradient(w,X,y):
    n,p = X.shape
    XT = X.T
    gradient = (2/n)*(XT @ (X @ w - y) ) +w
    return gradient

In [10]:
def norm(a):
    return np.linalg.norm(a)


In [11]:
def gradient_descent_1 (w,eita,X,y,limit):
    
    
    r_0 = norm(w)
    error = 0.001
    r_0_error = r_0 * error


    for i in range (0,limit) : 
        gradient_for_one = gradient(w,X,y)
        norm_for_one = norm(gradient_for_one)
        if (norm_for_one < r_0_error):
            break
        else :
            
            w = w - eita * gradient_for_one;
            
    return w

In [12]:
eita_list = [0.01,0.001,0.0001,0.00001,0.000001,0.0000001]

In [13]:
for i in range(6):
    print("For ", eita_list[i] )
    print(gradient_descent_1(initial_value,eita_list[i],X,y,200))

For  0.01
[[-0.03584202]
 [-0.00578563]
 [-0.03303181]
 [-0.03835415]
 [-0.02203995]
 [-0.14006302]
 [-0.06934711]
 [-0.0669861 ]
 [-0.05440737]
 [-0.32008189]
 [-0.00627788]
 [ 0.30180186]]
For  0.001
[[-0.0486647 ]
 [-0.00655425]
 [-0.0600501 ]
 [-0.06240208]
 [-0.0061649 ]
 [-0.10053875]
 [-0.0347275 ]
 [-0.08914474]
 [-0.04227347]
 [-0.14432854]
 [ 0.0601646 ]
 [ 0.15205827]]
For  0.0001
[[-0.04330552]
 [ 0.00546534]
 [-0.03354192]
 [-0.03289586]
 [ 0.0263771 ]
 [-0.04994577]
 [ 0.01617251]
 [-0.05779627]
 [ 0.00367169]
 [ 0.02275041]
 [ 0.03638957]
 [ 0.00392614]]
For  1e-05
[[-0.04094556]
 [ 0.00858591]
 [-0.0252464 ]
 [-0.02403713]
 [ 0.03483165]
 [-0.04007967]
 [ 0.02541883]
 [-0.04964782]
 [ 0.01320526]
 [ 0.04708432]
 [ 0.02901403]
 [-0.01981593]]
For  1e-06
[[-0.04067653]
 [ 0.00893122]
 [-0.02431755]
 [-0.02304706]
 [ 0.0357693 ]
 [-0.03901294]
 [ 0.02640984]
 [-0.048748  ]
 [ 0.01423942]
 [ 0.04961965]
 [ 0.02819396]
 [-0.0223217 ]]
For  1e-07
[[-0.04064927]
 [ 0.00896611]

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
from sklearn.metrics import mean_squared_error


def MSE_for_ridge(X_train, X_test, y_train, y_test):
    ridge = Ridge(alpha=1.0)
    ridge.fit(X_train,y_train)
    y_pred = ridge.predict(X_test) 
    n = len(X_test)
    mse = sum((y_pred - y_test) ** 2)/n
    
    return mse

In [16]:
# create training and testing vars
from sklearn.model_selection import RepeatedKFold
kf = RepeatedKFold(n_splits=5, n_repeats=1, random_state=None) 

mse_score = []
for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index] 
    y_train, y_test = y[train_index], y[test_index]
    
    

    
    MSE_for_one =  MSE_for_ridge(X_train, X_test, y_train, y_test )
    
    mse_score.append(MSE_for_one)

In [17]:
mse_score

[array([0.29540354]),
 array([0.31154248]),
 array([0.27273623]),
 array([0.24698233]),
 array([0.30989675])]

In [18]:
sum(mse_score)/len(mse_score)

array([0.28731227])

In [19]:
def file_processor (filename):
    X , y = datasets.load_svmlight_file(filename)
    X = sparse.csr_matrix(X)
    y = np.matrix(y).T
    y = sparse.csr_matrix(y)
    return X,y

In [20]:
X_train_ori,y_train_1_3 = file_processor("/Users/yilanliu/Desktop/UC DAVIS/STA 141C/HW1/E2006.train")
X_test_1_3,y_test_1_3 = file_processor("/Users/yilanliu/Desktop/UC DAVIS/STA 141C/HW1/E2006.test")

In [21]:
X_train_ori.shape

(16087, 150360)

In [22]:
X_test_1_3.shape

(3308, 150358)

Notice that the test and training matrix are different, thus we decide to delete the last two colunm of train matrix to match with test matrix.

In [23]:
X_train_1_3 = X_train_ori[:, 0: 150358]

In [24]:
n, p = X_train_1_3.shape

In [25]:
def mse_difficult(initial_value, eita,method):
    w_train_1_3 = method (initial_value,eita,X_train_1_3,y_train_1_3,200)
    w_train_1_3 = sparse.csr_matrix(w_train_1_3)
    y_pred_1_3 = (X_test_1_3) @ w_train_1_3
    mse = np.mean((y_pred_1_3 - y_test_1_3).data**2)
    return mse

In [26]:
initial_value_1_3 = np.transpose(np.matrix(np.random.uniform(-0.05,0.05,p)))


In [27]:
mse_difficult(initial_value_1_3,0.0001,gradient_descent_1)

5.477064099185314

In [29]:
for i in range(6):
    print(mse_difficult(initial_value_1_3,eita_list[i],gradient_descent_1))

0.1508845706493993
0.15521797807877688
5.477064099185314
12.833994732645554
13.988969433530038
14.110144416707044


In [30]:
X,y = file_processor("/Users/yilanliu/Desktop/UC DAVIS/STA 141C/HW1/news20.binary")

In [31]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, test_size=0.2)

In [32]:
X.shape

(19996, 1355191)

In [33]:
n_tr, p = X_train_2.shape

In [34]:
def gradient_logit(w,X,y):
    yXt = -(y.multiply(X)).T
    yXW = y.multiply(X) @ w
    np.exp(yXW.data, out = yXW.data)
    yXW.data = 1 / (yXW.data + 1)
    gradient =   yXt @ yXW + w
    return gradient

In [35]:
def gradient_descent_2 (w,eita,X,y,limit):
    X = X_train_2
    y = y_train_2
    r_0 = np.sqrt(sum(w.data**2))
    error = 0.001
    r_0_error = r_0 * error


    for i in range (0,limit) :
        gradient_for_one = gradient_logit(w,X,y)
        norm_for_one = np.sqrt(sum(gradient_for_one.data**2))
        if (norm_for_one < r_0_error):
            break
        else :
            w = w - eita * gradient_for_one;
            
    return w

In [36]:
initial_value_2 = np.matrix(np.random.uniform(-0.05,0.05,p)).T
initial_value_2 = sparse.csr_matrix(initial_value_2)

In [113]:
trial_1 = gradient_descent_2 (initial_value_2,0.01,X,y,200)

In [115]:
trial_1.todense(order=None, out=None)

matrix([[ 0.03548833],
        [ 0.21453628],
        [ 0.36395045],
        ...,
        [-0.06674191],
        [-0.06551046],
        [-0.04206018]])

In [37]:
from sklearn.metrics import accuracy_score

In [38]:
def accuarcy_2(eita,x_te, y_te):
    w = gradient_descent_2 (initial_value_2,eita,X,y,200)
    y_pred = sparse.csr_matrix.todense(x_te @ w)
    y_pred[ y_pred > 0] =  1
    y_pred[ y_pred < 0] = -1
    n,b = y_te.shape
    y_te = sparse.csr_matrix.todense(y_te)
    accuarcy = accuracy_score(y_te,y_pred,normalize=False)
    return accuarcy /n
    
    

In [39]:
accuarcy_2(0.01,X_test_2, y_test_2)

0.928

In [40]:
for i in range(6):
    print(accuarcy_2(eita_list[i],X_test_2, y_test_2))

0.928
0.879
0.8155
0.739
0.53625
0.5015
